In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from feature_engine.imputation import (AddMissingIndicator, 
                                       MeanMedianImputer, 
                                       CategoricalImputer
                                       )
from feature_engine.encoding import RareLabelEncoder, OneHotEncoder, OrdinalEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
import joblib

## Leer data gold

In [2]:
data_gold = pd.read_excel("../data/gold/car_gold.xlsx")
## selected feature
features = ["price", "antiguedad", "kilometraje", "vehicle_brand", "vehicle_line", 'location_state']
data_gold = data_gold[features]
data_gold.head()

,price,antiguedad,kilometraje,vehicle_brand,vehicle_line,location_state
0,183900000,0.0,5800.0,byd,song,bogota_d.c.
1,64900000,8.0,59000.0,mazda,3,antioquia
2,72000000,6.0,47000.0,NaN,NaN,bogota_d.c.
3,59500000,0.0,3600.0,kia,picanto,bogota_d.c.
4,63000000,9.0,81000.0,nissan,x-trail,bogota_d.c.


In [3]:
## dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(data_gold.drop(labels=['price'], axis=1),
                                                    data_gold['price'],
                                                    test_size=0.3,
                                                    random_state=0) # reproducibilidad

In [4]:
print(X_train.shape)
print(X_test.shape)
print(y_train)
print(y_test)

(5075, 5)
(2175, 5)
6835     22000000
6809     61000000
5029     33500000
2423     96000000
2434     37000000
          ...    
4931     59000000
3264    135000000
1653     70000000
2607     34600000
2732    109000000
Name: price, Length: 5075, dtype: int64
4178    188500000
528      38000000
7233     70000000
4658    169000000
4281     50000000
          ...    
3861     41000000
2484     39000000
2753     59900000
1941     75000000
5492     71900000
Name: price, Length: 2175, dtype: int64


## missing indicator

In [5]:
vars_with_na = [var for var in data_gold.columns if data_gold[var].isnull().sum() > 0]
indicator = AddMissingIndicator(variables=vars_with_na)
indicator.fit(X_train)
X_train_transformed = indicator.transform(X_train)
X_train_transformed.head()

,antiguedad,kilometraje,vehicle_brand,vehicle_line,location_state,antiguedad_na,kilometraje_na,vehicle_brand_na,vehicle_line_na,location_state_na
6835,16.0,109481.0,renault,symbol,bogota_d.c.,0,0,0,0,0
6809,16.0,269000.0,chevrolet,luv,bogota_d.c.,0,0,0,0,0
5029,11.0,163700.0,seat,ibiza,bogota_d.c.,0,0,0,0,0
2423,6.0,18000.0,mercedes-benz,clase,bogota_d.c.,0,0,0,0,0
2434,NaN,233000.0,suzuki,grand,antioquia,1,0,0,0,0


## Variables numericas con NA

In [6]:
num_vars = data_gold.select_dtypes(include=['int64', 'float64']).columns
num_vars_na = [var for var in num_vars if var in vars_with_na]
numerical_imputer = MeanMedianImputer(imputation_method='median', 
                                      variables=num_vars_na)

numerical_imputer.fit(X_train_transformed)
X_train_transformed = numerical_imputer.transform(X_train_transformed)


## Variables categoricas con NA

In [7]:
cat_vars = data_gold.select_dtypes(include=['object']).columns
cat_vars_na = [var for var in cat_vars if var in vars_with_na]
categorical_imputer = CategoricalImputer(imputation_method='missing',
                                        fill_value='missing',
                                        variables=cat_vars_na)
categorical_imputer.fit(X_train_transformed)
X_train_transformed = categorical_imputer.transform(X_train_transformed)
X_train_transformed.head()

,antiguedad,kilometraje,vehicle_brand,vehicle_line,location_state,antiguedad_na,kilometraje_na,vehicle_brand_na,vehicle_line_na,location_state_na
6835,16.0,109481.0,renault,symbol,bogota_d.c.,0,0,0,0,0
6809,16.0,269000.0,chevrolet,luv,bogota_d.c.,0,0,0,0,0
5029,11.0,163700.0,seat,ibiza,bogota_d.c.,0,0,0,0,0
2423,6.0,18000.0,mercedes-benz,clase,bogota_d.c.,0,0,0,0,0
2434,7.0,233000.0,suzuki,grand,antioquia,1,0,0,0,0


## Codificando raros

In [8]:
rare_encoder = RareLabelEncoder(tol=0.001, variables=cat_vars.tolist(), n_categories=1)
rare_encoder.fit(X_train_transformed)
X_train_transformed = rare_encoder.transform(X_train_transformed)
X_train_transformed.head()

,antiguedad,kilometraje,vehicle_brand,vehicle_line,location_state,antiguedad_na,kilometraje_na,vehicle_brand_na,vehicle_line_na,location_state_na
6835,16.0,109481.0,renault,symbol,bogota_d.c.,0,0,0,0,0
6809,16.0,269000.0,chevrolet,luv,bogota_d.c.,0,0,0,0,0
5029,11.0,163700.0,seat,Rare,bogota_d.c.,0,0,0,0,0
2423,6.0,18000.0,mercedes-benz,clase,bogota_d.c.,0,0,0,0,0
2434,7.0,233000.0,suzuki,grand,antioquia,1,0,0,0,0


## Codificando variables categoricas

### One hot encoder

In [10]:
one_hot = OneHotEncoder(variables=cat_vars.tolist(), drop_last=True)
## averiguar que otros metodos de codificacion existen y aplicar el que mas les guste 
one_hot.fit(X_train_transformed)
one_hot.transform(X_train_transformed)

,antiguedad,kilometraje,antiguedad_na,kilometraje_na,vehicle_brand_na,vehicle_line_na,location_state_na,vehicle_brand_renault,vehicle_brand_chevrolet,vehicle_brand_seat,...,vehicle_line_golf,vehicle_line_focus,vehicle_line_niro,vehicle_line_outlander,vehicle_line_blazer,vehicle_line_odyssey,location_state_bogota_d.c.,location_state_antioquia,location_state_missing,location_state_cundinamarca
6835,16.0,109481.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
6809,16.0,269000.0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
5029,11.0,163700.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2423,6.0,18000.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2434,7.0,233000.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931,17.0,65000.0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3264,7.0,25036.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1653,4.0,36000.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2607,11.0,110000.0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [13]:
ordinal_encoder = OrdinalEncoder(variables=cat_vars.tolist(), encoding_method='ordered')

In [15]:
ordinal_encoder.fit(X_train_transformed, y_train)

,encoding_method,'ordered'
,variables,"['vehicle_brand', 'vehicle_line', ...]"
,missing_values,'raise'
,ignore_format,False
,unseen,'ignore'


In [18]:
ordinal_encoder.encoder_dict_

{'vehicle_brand': {'skoda': 0,
  'fiat': 1,
  'ssangyong': 2,
  'renault': 3,
  'chevrolet': 4,
  'volkswagen': 5,
  'ram': 6,
  'dodge': 7,
  'hyundai': 8,
  'seat': 9,
  'kia': 10,
  'citroen': 11,
  'suzuki': 12,
  'honda': 13,
  'nissan': 14,
  'peugeot': 15,
  'ford': 16,
  'missing': 17,
  'mitsubishi': 18,
  'Rare': 19,
  'mazda': 20,
  'mini': 21,
  'subaru': 22,
  'jeep': 23,
  'audi': 24,
  'mercedes-benz': 25,
  'bmw': 26,
  'volvo': 27,
  'land': 28,
  'toyota': 29,
  'byd': 30,
  'porsche': 31},
 'vehicle_line': {'symbol': 0,
  'megane': 1,
  '206': 2,
  'twingo': 3,
  'optra': 4,
  'aveo': 5,
  'clio': 6,
  'spark': 7,
  'tiida': 8,
  'sail': 9,
  'fluence': 10,
  'lancer': 11,
  'bora': 12,
  'sonic': 13,
  'logan': 14,
  'n300': 15,
  'actyon': 16,
  'sandero': 17,
  'fiesta': 18,
  'accent': 19,
  'cruze': 20,
  'kwid': 21,
  'march': 22,
  'van': 23,
  'kangoo': 24,
  'focus': 25,
  'strada': 26,
  'crossfox': 27,
  'gol': 28,
  'getz': 29,
  'picanto': 30,
  'jetta':

In [17]:
ordinal_encoder.transform(X_train_transformed)

,antiguedad,kilometraje,vehicle_brand,vehicle_line,location_state,antiguedad_na,kilometraje_na,vehicle_brand_na,vehicle_line_na,location_state_na
6835,16.0,109481.0,3,0,1,0,0,0,0,0
6809,16.0,269000.0,4,42,1,0,0,0,0,0
5029,11.0,163700.0,9,88,1,0,0,0,0,0
2423,6.0,18000.0,25,124,1,0,0,0,0,0
2434,7.0,233000.0,12,62,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
4931,17.0,65000.0,18,89,0,0,1,0,0,1
3264,7.0,25036.0,16,110,1,1,0,0,0,0
1653,4.0,36000.0,20,52,0,0,0,0,0,1
2607,11.0,110000.0,4,39,1,0,0,0,0,0


In [9]:
#(x - mean)/std --> media 0 y desviacion estandar 1
#(x - min)/(max - min) --> min 0 y max 1

## Escalado de datos

In [11]:
scaler = SklearnTransformerWrapper(transformer=MinMaxScaler())
scaler.fit(X_train_transformed)
X_train_transformed = scaler.transform(X_train_transformed)

In [12]:
X_train_transformed.head()

,antiguedad,kilometraje,antiguedad_na,kilometraje_na,vehicle_brand_na,vehicle_line_na,location_state_na,vehicle_brand_renault,vehicle_brand_chevrolet,vehicle_brand_seat,...,location_state_bogota_d.c.,location_state_antioquia,location_state_missing,location_state_cundinamarca,location_state_quindio,location_state_valle_del_cauca,location_state_meta,location_state_santander,location_state_tolima,location_state_boyaca
6835,0.275862,0.127535,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6809,0.275862,0.313361,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5029,0.189655,0.190695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2423,0.103448,0.020967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2434,0.120690,0.271424,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Pongamos todo en un pipeline

In [20]:
pipeline_steps = [
    ("missing_indicator", AddMissingIndicator(variables=vars_with_na)),
    ("numerical_imputation", MeanMedianImputer(imputation_method='median', variables=num_vars_na)),
    ("categorical_imputation", CategoricalImputer(imputation_method='missing', fill_value='missing', variables=cat_vars_na)),
    ("rare_label_encoder", RareLabelEncoder(tol=0.001, variables=cat_vars.tolist(), n_categories=1)),
    ("ordinal_encoder", OrdinalEncoder(variables=cat_vars.tolist(), encoding_method='ordered')),
    ("scaler", SklearnTransformerWrapper(transformer=MinMaxScaler()))
]

In [21]:
fe_pipeline = Pipeline(pipeline_steps)

In [23]:
fe_pipeline

,steps,"[('missing_indicator', ...), ('numerical_imputation', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,missing_only,True
,variables,"['antiguedad', 'kilometraje', ...]"
,imputation_method,'median'
,variables,"['antiguedad', 'kilometraje']"
,imputation_method,'missing'
,fill_value,'missing'
,variables,"['vehicle_brand', 'vehicle_line', ...]"


In [24]:
fe_pipeline.fit(X_train, y_train)

,steps,"[('missing_indicator', ...), ('numerical_imputation', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,missing_only,True
,variables,"['antiguedad', 'kilometraje', ...]"
,imputation_method,'median'
,variables,"['antiguedad', 'kilometraje']"
,imputation_method,'missing'
,fill_value,'missing'
,variables,"['vehicle_brand', 'vehicle_line', ...]"


In [25]:
X_train_transformed = fe_pipeline.transform(X_train)

In [26]:
X_test_transformed = fe_pipeline.transform(X_test)

In [27]:
X_train_transformed.head()

,antiguedad,kilometraje,vehicle_brand,vehicle_line,location_state,antiguedad_na,kilometraje_na,vehicle_brand_na,vehicle_line_na,location_state_na
6835,0.275862,0.127535,0.096774,0.000000,0.25,0.0,0.0,0.0,0.0,0.0
6809,0.275862,0.313361,0.129032,0.302158,0.25,0.0,0.0,0.0,0.0,0.0
5029,0.189655,0.190695,0.290323,0.633094,0.25,0.0,0.0,0.0,0.0,0.0
2423,0.103448,0.020967,0.806452,0.892086,0.25,0.0,0.0,0.0,0.0,0.0
2434,0.120690,0.271424,0.387097,0.446043,0.75,1.0,0.0,0.0,0.0,0.0


In [30]:
X_train_transformed.to_excel("../data/gold/X_train_transformed.xlsx", index=False)
X_test_transformed.to_excel("../data/gold/X_test_transformed.xlsx", index=False)

In [29]:
joblib.dump(fe_pipeline, "../models/fe_pipeline.joblib")

['../models/fe_pipeline.joblib']